In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [2]:
test_path_dir = '../data/testA'
test_dirs = os.listdir(test_path_dir)
for file in test_dirs:
    print(file)

totalExposureLog.out
mini_data
test_sample.dat
ad_static_feature.out
ad_operation.dat
user_data


In [3]:
# 一共有5个文件
# totalExposureLog.out:历史曝光日志数据文件
# user_data:用户特征属性文件
# ad_static_feature.out:广告静态数据文件
# ad_operation.dat:广告操作数据
# test_sample.dat:需要预测的文件

### 1.1 处理历史曝光日志数据

In [4]:
'''
    其中该文件有10列，每列描述如下：
    广告请求id:request_id
    广告请求时间:request_time
    广告位id:location_id
    用户id:user_id
    曝光广告id:ad_id
    曝光广告素材尺寸:ad_size
    曝光广告出价bid:ad_bid
    曝光广告pctr:ad_pctr
    曝光广告quality_ecpm:ad_quality_ecpm
    曝光广告totalEcpm:ad_totalEcpm
'''
names = ['request_id','request_time','location_id','user_id','ad_id','ad_size',\
         'ad_bid','ad_pctr','ad_quality_ecpm','ad_totalEcpm']

totalExposureLog_file = '../data/testA/mini_data/billion_totalExposureLog.out'
exposure_df = pd.read_csv(totalExposureLog_file,delimiter = '\t',\
                          parse_dates = ['request_time'],header=None,names = names)

In [5]:
# 分析题目 根据之前n天的数据来预测后一天的日曝光数据
# 首先需要将日志文件进行整理，整合成数据
exposure_df.head()

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320


In [6]:
exposure_df['request_datetime'] = pd.to_datetime(exposure_df['request_time'],unit='s')

In [7]:
exposure_df.head()

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm,request_datetime
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17 13:22:26
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17 02:34:52
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17 15:16:40
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17 01:11:38
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16 23:52:39


In [8]:
# 统计每列数据的缺失情况
num = exposure_df.isna().sum()

In [9]:
num

request_id          0
request_time        0
location_id         0
user_id             0
ad_id               0
ad_size             0
ad_bid              0
ad_pctr             0
ad_quality_ecpm     0
ad_totalEcpm        0
request_datetime    0
dtype: int64

In [10]:
# 可以看出曝光的历史数据文件不缺失任何数值，按照我的理解，数据都是可信的
# 接下来首先需要获取每天每个广告的曝光数量
exposure_df['request_day'] = exposure_df['request_datetime'].dt.strftime('%Y-%m-%d/')

In [11]:
exposure_df.head()

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm,request_datetime,request_day
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17 13:22:26,17/02/2019
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17 02:34:52,17/02/2019
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17 15:16:40,17/02/2019
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17 01:11:38,17/02/2019
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16 23:52:39,16/02/2019


In [12]:
group_df = exposure_df.groupby(['request_day','ad_id'])['request_id'].count()

In [13]:
group_df = group_df.reset_index()

In [14]:
group_df.head()

,request_day,ad_id,request_id
0,16/02/2019,61,3
1,16/02/2019,63,1
2,16/02/2019,104,1
3,16/02/2019,167,17
4,16/02/2019,184,2


In [15]:
exposure_df.loc[exposure_df['ad_id'] == 61]

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm,request_datetime,request_day
97201,72933177,1550360345,257,645852,61,30,152,2.378,47.56,409.016,2019-02-16 23:39:05,16/02/2019
101513,93605818,1550359387,328,1005827,61,30,48,0.406,8.12,27.608,2019-02-16 23:23:07,16/02/2019
140234,77865276,1550359485,25,1214464,61,30,77,6.119,122.38,593.543,2019-02-16 23:24:45,16/02/2019
236124,21247779,1550364914,328,46123,61,30,63,1.262,25.24,104.746,2019-02-17 00:55:14,17/02/2019
285831,87911304,1550362018,25,821851,61,30,24,5.115,102.30,225.060,2019-02-17 00:06:58,17/02/2019
538316,17559924,1550362105,25,830003,61,30,84,8.656,173.12,900.224,2019-02-17 00:08:25,17/02/2019
582301,1935367,1550362789,328,811101,61,30,69,0.956,19.12,85.084,2019-02-17 00:19:49,17/02/2019
595792,80461475,1550363717,328,232744,61,30,32,0.859,17.18,44.668,2019-02-17 00:35:17,17/02/2019
728010,48256753,1550362308,116,458929,61,30,41,0.998,0.00,40.918,2019-02-17 00:11:48,17/02/2019
758413,74717106,1550363900,328,1164597,61,30,72,1.981,39.62,182.252,2019-02-17 00:38:20,17/02/2019


In [16]:
# 这个能发现每个时刻的广告的数据也不同，即不同时刻的广告数据和不同的用户组合，该用户是否点击也不一定

In [17]:
# 查看日期
list(exposure_df['request_day'].unique())

['17/02/2019', '16/02/2019']

In [18]:
# 先想一个baseline,不考虑其他特征,只将日曝光数据作为特征，进行预测
group_df = group_df.rename(columns = {'request_id':'exp_num'})

In [19]:
group_df.head()

,request_day,ad_id,exp_num
0,16/02/2019,61,3
1,16/02/2019,63,1
2,16/02/2019,104,1
3,16/02/2019,167,17
4,16/02/2019,184,2


In [22]:
type(group_df['request_day'])

pandas.core.series.Series

In [39]:
def add_days(x,i):
    ss = datetime.strptime(x,'%d/%m/%Y') + timedelta(days = i)
    return ss.strftime('%d/%m/%Y')

In [40]:
def create_lagging(df,df_origin,i):
    df1 = df_origin.copy()
    df1['request_day'] = df1['request_day'].apply(add_days,args = (i,))
    df1 = df1.rename(columns = {'exp_num':'lagging' + str(i)})
    df2 = pd.merge(df,df1[['ad_id','request_day','lagging' + str(i)]],on = ['request_day','ad_id'],\
                  how = 'left')
    return df2

In [30]:
ss = datetime.strptime(stt, '%d/%m/%Y') + timedelta(days = 1)

In [32]:
ss.strftime('%d/%m/%Y')

'17/02/2019'

In [28]:
stt = group_df.loc[0,'request_day']

In [41]:
df1 = create_lagging(group_df, group_df, 1)
for i in range(2, 2 + 1):
    df1 = create_lagging(df1, group_df, i)

,request_day,ad_id,exp_num,lagging1,lagging2
0,16/02/2019,61,3,NaN,NaN
1,16/02/2019,63,1,NaN,NaN
2,16/02/2019,104,1,NaN,NaN
3,16/02/2019,167,17,NaN,NaN
4,16/02/2019,184,2,NaN,NaN
5,16/02/2019,308,1,NaN,NaN
6,16/02/2019,319,11,NaN,NaN
7,16/02/2019,347,6,NaN,NaN
8,16/02/2019,378,3,NaN,NaN
9,16/02/2019,476,2,NaN,NaN


In [44]:
group_df.groupby(['ad_id'])['exp_num'].sum()

ad_id
6           1
13          6
27          2
36          1
61         13
63          1
104         5
152         4
167       193
174         1
184        20
189        10
220         1
249         1
251         3
255         1
265         5
281         5
308         6
312         3
319        29
327         1
347        45
351        52
378         4
380         2
392         2
415         1
423        37
430         1
         ... 
735511     38
735522      1
735537      1
735539      3
735572      3
735613      5
735622      3
735663      1
735669      2
735682     56
735685      4
735748      7
735754      3
735763    114
735765      1
735811     10
735820     12
735821      2
735824      6
735835    479
735846      8
735884      6
735885      1
735897      5
735936      1
735955      2
735971      5
736015      2
736017      7
736018      9
Name: exp_num, Length: 38433, dtype: int64

In [77]:
dd = pd.read_csv('../data/testA/exposure_group_all.csv')

In [78]:
dd.head()

,request_day,ad_id,exp_num
0,2019-02-16,61,17.0
1,2019-02-16,63,1.0
2,2019-02-16,104,1.0
3,2019-02-16,167,84.0
4,2019-02-16,184,18.0


In [74]:
ll = len(list(dd.exp_num.unique()))

In [75]:
ll

4888

In [80]:
def exp_sum(group):
    group['ad_id_en'] = group['exp_num'].sum()
    return group

dd = dd.groupby('ad_id').apply(exp_sum)
sorted_link = np.sort(dd['ad_id_en'].unique())
dd['ad_id_en'] = dd['ad_id_en'].map(lambda x: np.argmin(x >= sorted_link))

In [ ]:
dd.boxplot(by=['ad_id_en'], column='exp_num')
plt.show()